In [1]:
!pip install -q  openml
!pip install -q  catboost
!pip install -q optuna
!pip install -q xgboost
!pip install -q lightgbm
!pip install -q PyGithub

# !nvidia-smi  # this should display information about available GPUs
# %load_ext cuml.accel

In [2]:
import openml
import os
import pandas as pd
import requests
from scipy.io import arff

# Load dataset metadata
df = openml.datasets.list_datasets(output_format='dataframe')

# Filter by GEMLeR: binary class, ≥10k features, and 'AP_' or 'OVA_' in name
gemler = df[
    (df['NumberOfClasses'] == 2) &
    (df['NumberOfFeatures'] >= 10000) &
    (df['name'].str.startswith(('AP_', 'OVA_')))
]
gemler['ImbalanceRatio']=gemler['MajorityClassSize']/gemler['MinorityClassSize']
gemler_sorted = gemler.sort_values(by='ImbalanceRatio',ascending=False).head(3)

# Save to CSV
gemler_sorted[['did', 'name', 'NumberOfInstances', 'NumberOfFeatures', 'MajorityClassSize', 'MinorityClassSize', 'ImbalanceRatio']].to_csv("GEMLeR_Datasets.csv", index=False)

print(gemler_sorted[['did', 'name', 'NumberOfInstances', 'NumberOfFeatures', 'MajorityClassSize', 'MinorityClassSize', 'ImbalanceRatio']])


       did             name  NumberOfInstances  NumberOfFeatures  \
1142  1142  OVA_Endometrium             1545.0           10936.0   
1146  1146     OVA_Prostate             1545.0           10937.0   
1139  1139      OVA_Omentum             1545.0           10936.0   

      MajorityClassSize  MinorityClassSize  ImbalanceRatio  
1142             1484.0               61.0       24.327869  
1146             1476.0               69.0       21.391304  
1139             1468.0               77.0       19.064935  


/tmp/ipykernel_3087/1149535004.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gemler['ImbalanceRatio']=gemler['MajorityClassSize']/gemler['MinorityClassSize']


In [3]:
def encode_target(df, dataset_name):
    target_col = df['Tissue']

    if dataset_name.startswith("AP_"):
        # Format: AP_Word1_Word2
        _, word1, word2 = dataset_name.split("_", 2)
        label_map = {word1: 1, word2: 0}
        y = target_col.map(label_map)

    elif dataset_name.startswith("OVA_"):
        # Format: OVA_Word1
        _, word1 = dataset_name.split("_", 1)
        y = target_col.apply(lambda x: 1 if x == word1 else 0)

    else:
        raise ValueError("Unknown dataset format")

    return y

# Output directory for ARFF files
arff_dir = "datasets_arff"
os.makedirs(arff_dir, exist_ok=True)

# Dictionary to store loaded DataFrames
datasets = {}

for _, row in gemler_sorted.iterrows():
    did = int(row['did'])
    name = row['name']

    # Create safe filename
    safe_name = name.replace(' ', '_').replace('/', '_')
    arff_path = os.path.join(arff_dir, f"{safe_name}.arff")

    # Download dataset metadata
    dataset = openml.datasets.get_dataset(did)

    # Download only if file does not already exist
    if not os.path.exists(arff_path):
        file_url = dataset.url
        response = requests.get(file_url)
        if response.status_code != 200:
            print(f"❌ Failed to download dataset '{name}' (ID {did})")
            continue
        with open(arff_path, 'wb') as f:
            f.write(response.content)
        print(f"✅ Downloaded and saved ARFF file: {arff_path}")
    else:
        print(f"🔁 Skipped download (already exists): {arff_path}")

    # Load the ARFF file into a DataFrame
    data, meta = arff.loadarff(arff_path)
    df = pd.DataFrame(data)
    # Decode byte columns to strings if needed
    for col in df.select_dtypes([object]).columns:
        df[col] = df[col].apply(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)
    df['Tissue']=encode_target(df, safe_name)
    # Store DataFrame
    datasets[safe_name] = df
    print(f"📊 Loaded DataFrame: {safe_name} with shape {df.shape}")

🔁 Skipped download (already exists): datasets_arff/OVA_Endometrium.arff
📊 Loaded DataFrame: OVA_Endometrium with shape (1545, 10937)
🔁 Skipped download (already exists): datasets_arff/OVA_Prostate.arff
📊 Loaded DataFrame: OVA_Prostate with shape (1545, 10937)
🔁 Skipped download (already exists): datasets_arff/OVA_Omentum.arff
📊 Loaded DataFrame: OVA_Omentum with shape (1545, 10937)


In [4]:


for safe_name, df in datasets.items():
    # Retrieve the original OpenML dataset using the name
    # (optional: you could cache the target name in earlier code)
    matching_row = gemler_sorted[gemler_sorted['name'].str.replace(' ', '_') == safe_name]

    if matching_row.empty:
        print(f"⚠️  Could not find metadata for: {safe_name}")
        continue

    did = int(matching_row['did'].values[0])
    dataset = openml.datasets.get_dataset(did)

    target_col = dataset.default_target_attribute
    if target_col not in df.columns:
        print(f"⚠️  Target column '{target_col}' not found in: {safe_name}")
        continue
    print(f"\n📊 Dataset: {safe_name} (Target: {target_col})")
    #print(df['Tissue'],encode_target(df, safe_name))
    print(df[target_col].value_counts())



📊 Dataset: OVA_Endometrium (Target: Tissue)
Tissue
0    1484
1      61
Name: count, dtype: int64

📊 Dataset: OVA_Prostate (Target: Tissue)
Tissue
0    1476
1      69
Name: count, dtype: int64

📊 Dataset: OVA_Omentum (Target: Tissue)
Tissue
0    1468
1      77
Name: count, dtype: int64


In [ ]:
import optuna
import numpy as np
from sklearn.model_selection import StratifiedKFold, cross_val_score, RepeatedStratifiedKFold, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score, make_scorer, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA, FastICA, TruncatedSVD
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from lightgbm import LGBMClassifier
from github import Github
import joblib
import random
import json
import pickle
import lzma


#calibrate classifier as well!!!
#Do brier score!!!?! and what else
#rewrite the CV score function to output several metrics!?


random.seed(42)
cv_results = {}

# Define outer CV (final model evaluation)
outer_cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=42)



#study_summaries = optuna.get_all_study_summaries(storage)

# Delete each study
#for summary in study_summaries:
#    optuna.delete_study(study_name=summary.study_name, storage=storage)

# List of models to compare
classifiers = {
 #    "Logistic Regression": LogisticRegression,
 ##   "LDA" : LinearDiscriminantAnalysis,
 ##   "QDA" : QuadraticDiscriminantAnalysis,
 # "Naive Bayes" : GaussianNB,
 # "SVM (Linear)" : SVC,
 ##   "SVM (RBF)": SVC,
 #  "k-NN": KNeighborsClassifier,
    "Random Forest": RandomForestClassifier,
 ##   "Gradient Boosting": GradientBoostingClassifier,
 #   "CatBoost": CatBoostClassifier,
 #    "XGBoost": XGBClassifier,
 #   "LightGBM": LGBMClassifier
}

# Balanced accuracy scorer
auc_scorer = make_scorer(roc_auc_score)

# Store best models and scores
best_models = {}
best_scores = {}

# Objective builders for each model
def build_objective(model_name, X, y, dr_method):
    def objective(trial):
        steps=[('scaler', StandardScaler())]
        if dr_method == "pca":
            ncomp = trial.suggest_int("ncomp", 5, min(X.shape[1],0.8*X.shape[0]))
            dr_step = ('dr', PCA(n_components=ncomp, random_state=42))
        elif dr_method == "ica":
            ncomp = trial.suggest_int("ncomp", 5, min(X.shape[1], 0.8*X.shape[0]))
            dr_step = ('dr', FastICA(n_components=ncomp, whiten='unit-variance', max_iter=1000, random_state=42))
        elif dr_method == "svd":
            ncomp = trial.suggest_int("ncomp", 5, min(X.shape[1], 0.8*X.shape[0]))
            dr_step = ('dr', TruncatedSVD(n_components=ncomp, random_state=42))
        elif dr_method:
            raise ValueError("Unknown DR method")
        if dr_method:
            steps.append(dr_step)

        if model_name == "Logistic Regression":
            C = trial.suggest_float("C", 1e-4, 1e2,log=True)
            clf = LogisticRegression(penalty='l1', C=C, solver="liblinear", max_iter=1000)
            steps.append(('clf', clf))

        elif model_name == "LDA":
            solver = trial.suggest_categorical("solver", ["lsqr", "eigen"])
            shrinkage_opt = trial.suggest_categorical("shrinkage", ["None", "auto"])
            if shrinkage_opt == "None":
                shrinkage = None
            elif shrinkage_opt == "auto":
                shrinkage = "auto"
            # Suggest tol (used only with lsqr and eigen, but safe to include)
            tol = trial.suggest_float("tol", 1e-5, 1e-1, log=True)
            # Suggest n_components (must be <= min(n_classes-1, n_features), so use fixed upper bound)
            # Build classifier
            clf = LinearDiscriminantAnalysis(solver=solver, shrinkage=shrinkage,tol=tol)
            steps.append(('clf', clf))

        elif model_name == "QDA":
            reg_param = trial.suggest_float("reg_param", 0.0, 1)
            clf = QuadraticDiscriminantAnalysis(reg_param=reg_param)
            steps.append(('clf', clf))

        elif model_name == "Naive Bayes":
            var_smoothing = trial.suggest_float("var_smoothing", 1e-11, 1e-7,log=True)
            clf = GaussianNB(var_smoothing=var_smoothing)
            steps.append(('clf', clf))

        elif model_name == "SVM (Linear)":
            C = trial.suggest_float("C", 1e-4, 1e2,log=True)
            clf = SVC(kernel='linear', C=C, probability=True)
            steps.append(('clf', clf))

        elif model_name == "SVM (RBF)":
            C = trial.suggest_float("C", 1e-4, 1e2,log=True)
            gamma = trial.suggest_float("gamma", 1e-5, 1e-1,log=True)
            clf = SVC(kernel='rbf', C=C, gamma=gamma, probability=True)
            steps.append(('clf', clf))

        elif model_name == "k-NN":
            n_neighbors = trial.suggest_int("n_neighbors", 3, 30)
            weights = trial.suggest_categorical("weights", ["uniform", "distance"])
            clf = KNeighborsClassifier(n_neighbors=n_neighbors, weights=weights, n_jobs=-1)
            steps.append(('clf', clf))

        elif model_name == "Random Forest":
            n_estimators = trial.suggest_int("n_estimators", 100, 1000)
            max_depth = trial.suggest_int("max_depth", 2, 20)
            #min_samples_split = trial.suggest_int("min_samples_split", 5, 30)
            #min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 20)
            clf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, n_jobs=-1)
            steps.append(('clf', clf))

        elif model_name == "Gradient Boosting":
            learning_rate = trial.suggest_float("learning_rate", 0.01, 0.3)
            n_estimators = trial.suggest_int("n_estimators", 100, 1000)
            max_depth = trial.suggest_int("max_depth", 2, 10)
            clf = GradientBoostingClassifier(
                learning_rate=learning_rate, n_estimators=n_estimators, max_depth=max_depth, n_jobs=-1)
            steps.append(('clf', clf))

        elif model_name == "CatBoost":
            learning_rate = trial.suggest_float("learning_rate", 0.01, 0.3)
            depth = trial.suggest_int("depth", 4, 10)
            iterations = trial.suggest_int("iterations", 100, 1000)
#            l2_leaf_reg = trial.suggest_float('l2_leaf_reg', 1, 10),
#            border_count = trial.suggest_int('border_count', 32, 255),
            clf = CatBoostClassifier(learning_rate=learning_rate,depth=depth,iterations=iterations, early_stopping_rounds=20,verbose=0,thread_count=-1) #l2_leaf_reg=l2_leaf_reg,border_count = border_count,
            steps.append(('clf', clf))

        elif model_name == "XGBoost":
            learning_rate = trial.suggest_float("learning_rate", 0.01, 0.3)
            max_depth = trial.suggest_int("max_depth", 3, 10)
            n_estimators = trial.suggest_int("n_estimators", 100, 1000)
            clf = XGBClassifier(learning_rate=learning_rate, max_depth=max_depth, n_estimators=n_estimators, eval_metric='logloss', n_jobs=-1)
            steps.append(('clf', clf))

        elif model_name == "LightGBM":
            learning_rate = trial.suggest_float("learning_rate", 0.01, 0.3)
            max_depth = trial.suggest_int("max_depth", 3, 15)
            n_estimators = trial.suggest_int("n_estimators", 100, 1000)
            clf = LGBMClassifier(learning_rate=learning_rate, max_depth=max_depth, n_estimators=n_estimators,force_col_wise=True, deterministic=True, n_jobs=-1)
            steps.append(('clf', clf))

        else:
            raise ValueError(f"Model '{model_name}' not supported.")

        pipeline = Pipeline(steps)
        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        scores = cross_val_score(pipeline, X, y, cv=cv, scoring='roc_auc', n_jobs=-1)
        return np.mean(scores)
    return objective

# Main loop: for each model, tune + evaluate with outer CV
def run_nested_cv(X, y, classifiers,ds_name):
    for dr_method in [None,"pca", "ica", "svd"]:
        for name, model_cls in classifiers.items():
            print(f"\n🔍 Optimizing: {name}")
            outer_scores = []
            best_pipeline = None
            best_score = -np.inf
    #!!!! should swap the data loop with the classifer loop...
    #Will save on dim red computation!!!
            storage = optuna.storages.RDBStorage(url="sqlite:///"+name+"_study.db", engine_kwargs={"connect_args": {"timeout": 100}})


            for fold_idx, (train_idx, test_idx) in enumerate(outer_cv.split(X, y)):
                print(f"  Fold {fold_idx + 1}")

                X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
                y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

                #X_train, X_test = X[train_idx], X[test_idx]
                #y_train, y_test = y[train_idx], y[test_idx]

                study = optuna.create_study(study_name=ds_name+name+str(dr_method)+str(fold_idx)+str(random.randint(0, 1e6)),direction="maximize", storage=storage,load_if_exists=True)

                #"Random Forest",
                #if name in ["Gradient Boosting", "CatBoost","LightGBM"]:
                #    study.optimize(build_objective(name, X_train, y_train, dr_method), n_trials=1000, n_jobs=-1)
                #else:
                study.optimize(build_objective(name, X_train, y_train, dr_method), n_trials=300, n_jobs=-1)

                best_params = study.best_params
                dr_step = None

                if dr_method == "pca":
                    pca_n = best_params.pop("ncomp")
                    dr_step = ('dr', PCA(n_components=pca_n,random_state=42))

                elif dr_method == "ica":
                    ica_n = best_params.pop("ncomp")
                    dr_step = ('dr', FastICA(n_components=ica_n, whiten='unit-variance', max_iter=1000,random_state=42))

                elif dr_method == "svd":
                    svd_n = best_params.pop("ncomp")
                    dr_step = ('dr', TruncatedSVD(n_components=svd_n,random_state=42))

                steps = [('scaler', StandardScaler())]
                if dr_step:
                    steps.append(dr_step)

                dr_keys = ["ncomp"]
                # Filtered dict with only classifier params
                clf_params = {k: v for k, v in best_params.items() if k not in dr_keys}


                # Refit model with best params on train set
                if name == "Logistic Regression":
                    clf = model_cls(**clf_params, solver="liblinear", penalty='l1', max_iter=1000)
                    steps.append(('clf', clf))
                elif name == "LDA":
                    clf = model_cls(**clf_params)
                    steps.append(('clf', clf))
                elif name == "QDA":
                    clf = model_cls(**clf_params)
                    steps.append(('clf', clf))
                elif name == "Naive Bayes":
                    clf = model_cls(**clf_params)
                    steps.append(('clf', clf))
                elif name == "SVM (Linear)":
                    clf = model_cls(**clf_params,kernel='linear', probability=True)
                    steps.append(('clf', clf))
                elif name == "SVM (RBF)":
                    clf = model_cls(**clf_params,kernel='rbf', probability=True)
                    steps.append(('clf', clf))
                elif name == "k-NN":
                    clf = model_cls(**clf_params,n_jobs=-1)
                    steps.append(('clf', clf))
                elif name == "Random Forest":
                    clf = model_cls(**clf_params,n_jobs=-1)
                    steps.append(('clf', clf))
                elif name == "Gradient Boosting":
                    clf = model_cls(**clf_params)
                    steps.append(('clf', clf))
                elif name == "XGBoost":
                    clf = model_cls(**clf_params, eval_metric='logloss', n_jobs=-1)
                    steps.append(('clf', clf))
                elif name == "CatBoost":
                    clf = model_cls(**best_params, early_stopping_rounds=20,verbose=0,thread_count=-1,random_state=42)
                    steps.append(('clf', clf))
                elif name == "XGBoost":
                    clf =model_cls(**best_params, use_label_encoder=False, eval_metric="logloss", n_jobs=-1)
                    steps.append(('clf', clf))
                elif name == "LightGBM":
                    clf = model_cls(**best_params,force_col_wise=True, deterministic=True, n_jobs=-1)
                    steps.append(('clf', clf))

                pipeline = Pipeline(steps)
                pipeline.fit(X_train, y_train)
                y_pred = pipeline.predict_proba(X_test)[:, 1]
                score = roc_auc_score(y_test, y_pred)

                print(f"    AUC: {score:.4f}")
                outer_scores.append(score)

                if score > best_score:
                    best_score = score
                    best_pipeline = pipeline
                    mybest_params = best_params
                # --- Step 1: Authenticate with a personal access token (create one at https://github.com/settings/tokens)
                #g = Github("XXXX")

                # --- Step 2: Choose the repository
                #repo = g.get_user().get_repo("Data588")  # or use g.get_repo("username/repo")

                # --- Step 3: Prepare the file details
                #file_path = "example_study.db"
                #github_path = "experiments/HPClineardimred"+name+"/example_study.db"  # path in the repo

                #try:
                #    with open(file_path, "r", encoding="utf-8") as file:
                #        content = file.read()
                #except UnicodeDecodeError:
                #    with open(file_path, "r", encoding="latin1") as file:
                #        content = file.read()
                # --- Step 5: Upload (create or update the file)
                #try:
                #    contents = repo.get_contents(github_path)
                #    repo.update_file(contents.path, "Update file via script", content, contents.sha)
                #    print("File updated.")
                #except Exception:
                #    repo.create_file(github_path, "Initial commit via script", content)
                #    print("File uploaded.")

            # Save best model
            best_models[name] = best_pipeline
            best_scores[name] = outer_scores
            
            # Save with maximum compression
            #with lzma.open("best_model_"+ds_name+name.replace(' ', '_')+str(dr_method)+".pkl.xz", "wb", preset=9 | lzma.PRESET_EXTREME) as f:
            #    joblib.dump(best_pipeline, f)
    
            #print(f"✅ Saved best model for {name} with AUC: {np.mean(outer_scores):.4f}")
            #print(mybest_params)
            #print(model_cls)

            # --- Step 1: Authenticate with a personal access token (create one at https://github.com/settings/tokens)
            g = Github("XXXXX")

            # --- Step 2: Choose the repository
            repo = g.get_user().get_repo("Data588")  # or use g.get_repo("username/repo")

            # --- Step 3: Prepare the file details
            #file_path = "best_model_"+ds_name+name.replace(' ', '_')+str(dr_method)+".pkl.xz"
            #github_path = "experiments/HPClineardimred"+name+"/best_model_"+ds_name+name.replace(' ', '_')+str(dr_method)+".pkl.xz"  # path in the repo

            #try:
            #    with open(file_path, "r", encoding="utf-8") as file:
            #        content = file.read()
            #except UnicodeDecodeError:
            #    with open(file_path, "r", encoding="latin1") as file:
            #        content = file.read()
            # --- Step 5: Upload (create or update the file)
            #try:
            #    contents = repo.get_contents(github_path)
            #    repo.update_file(contents.path, "Update file via script", content, contents.sha)
            #    print("File updated.")
            #except Exception:
            #    repo.create_file(github_path, "Initial commit via script", content)
            #    print("File uploaded.")


            file_path = name+"_study.db"
            github_path = "experiments/HPClineardimred"+name+"/"+name+"_study.db"  # path in the repo
    
            with open(file_path, "rb") as file:
                content = file.read()

            # --- Step 5: Upload (create or update the file)
            try:
                contents = repo.get_contents(github_path)
                repo.update_file(contents.path, "Update file via script", content, contents.sha)
                print("File updated.")
            except Exception:
                repo.create_file(github_path, "Initial commit via script", content)
                print("File uploaded.")


            json_data = {
                "scores": outer_scores,
                "mean": float(np.mean(outer_scores)),
                "std": float(np.std(outer_scores)),
                "best_params": mybest_params
            }
            
            filename = f"{ds_name}_{name}_{dr_method}_cv_results.json"
            with open(filename, "w") as f:
                json.dump(json_data, f, indent=2)


            file_path = filename
            github_path = "experiments/HPClineardimred"+name+"/"+filename  # path in the repo

            try:
                with open(file_path, "r", encoding="utf-8") as file:
                    content = file.read()
            except UnicodeDecodeError:
                with open(file_path, "r", encoding="latin1") as file:
                    content = file.read()
            # --- Step 5: Upload (create or update the file)
            try:
                contents = repo.get_contents(github_path)
                repo.update_file(contents.path, "Update file via script", content, contents.sha)
                print("File updated.")
            except Exception:
                repo.create_file(github_path, "Initial commit via script", content)
                print("File uploaded.")

                
# Example usage
#from sklearn.datasets import load_breast_cancer
#X, y = load_breast_cancer(return_X_y=True)

#X, y = datasets["OVA_Endometrium"].drop(columns='Tissue'), datasets["OVA_Endometrium"]['Tissue']

#X = X[np.random.choice(X.columns, size=int(0.01 * X.shape[1]), replace=False)]
#X_reduced, _, y_reduced, _ = train_test_split(
#    X, y,
#    test_size=0.8,      # Keep 10%, drop 90%
#    stratify=y,
#    random_state=42     # For reproducibility)

## Add loop to run over all data sets!!!! (needs to be a parameter to run_nested and also needs to be stored in the study name of optuna!)

for safe_name, df in datasets.items():
    if safe_name != 'OVA_Omentum':
        X, y = df.drop(columns='Tissue'), df['Tissue']
#  run_nested_cv(X_reduced, y_reduced, classifiers, safe_name)
        run_nested_cv(X, y, classifiers, safe_name)






[I 2025-08-10 22:35:41,626] A new study created in RDB with name: OVA_EndometriumRandom ForestNone0670487



🔍 Optimizing: Random Forest
  Fold 1


[I 2025-08-10 22:35:59,626] Trial 7 finished with value: 0.924198371331655 and parameters: {'n_estimators': 173, 'max_depth': 17}. Best is trial 7 with value: 0.924198371331655.
[I 2025-08-10 22:36:05,463] Trial 2 finished with value: 0.9258431136167546 and parameters: {'n_estimators': 418, 'max_depth': 4}. Best is trial 2 with value: 0.9258431136167546.
[I 2025-08-10 22:36:07,255] Trial 9 finished with value: 0.9257704854093536 and parameters: {'n_estimators': 279, 'max_depth': 17}. Best is trial 2 with value: 0.9258431136167546.
[I 2025-08-10 22:36:12,360] Trial 15 finished with value: 0.924166223451406 and parameters: {'n_estimators': 526, 'max_depth': 8}. Best is trial 2 with value: 0.9258431136167546.
[I 2025-08-10 22:36:12,457] Trial 17 finished with value: 0.9359592951104492 and parameters: {'n_estimators': 931, 'max_depth': 3}. Best is trial 17 with value: 0.9359592951104492.
[I 2025-08-10 22:36:12,555] Trial 28 finished with value: 0.9277076906712052 and parameters: {'n_estima

    AUC: 0.9607
  Fold 2


[I 2025-08-10 22:41:57,973] Trial 8 finished with value: 0.9222545473885757 and parameters: {'n_estimators': 111, 'max_depth': 16}. Best is trial 8 with value: 0.9222545473885757.
[I 2025-08-10 22:42:04,531] Trial 1 finished with value: 0.9216969825905045 and parameters: {'n_estimators': 419, 'max_depth': 12}. Best is trial 8 with value: 0.9222545473885757.
[I 2025-08-10 22:42:04,932] Trial 14 finished with value: 0.9209389663038211 and parameters: {'n_estimators': 195, 'max_depth': 20}. Best is trial 8 with value: 0.9222545473885757.
[I 2025-08-10 22:42:11,825] Trial 21 finished with value: 0.9222640617428406 and parameters: {'n_estimators': 325, 'max_depth': 12}. Best is trial 21 with value: 0.9222640617428406.
[I 2025-08-10 22:42:13,512] Trial 16 finished with value: 0.9312443120708199 and parameters: {'n_estimators': 743, 'max_depth': 4}. Best is trial 16 with value: 0.9312443120708199.
[I 2025-08-10 22:42:16,881] Trial 5 finished with value: 0.9300719781583519 and parameters: {'n_

...

    AUC: 0.9985
  Fold 4


[I 2025-08-11 00:54:15,436] Trial 20 finished with value: 0.9754443707618992 and parameters: {'ncomp': 18, 'n_estimators': 738, 'max_depth': 14}. Best is trial 20 with value: 0.9754443707618992.
[I 2025-08-11 00:54:17,054] Trial 39 finished with value: 0.9662097807077424 and parameters: {'ncomp': 202, 'n_estimators': 259, 'max_depth': 10}. Best is trial 20 with value: 0.9754443707618992.
[I 2025-08-11 00:54:19,420] Trial 12 finished with value: 0.9699711662863347 and parameters: {'ncomp': 244, 'n_estimators': 863, 'max_depth': 15}. Best is trial 20 with value: 0.9754443707618992.
[I 2025-08-11 00:54:19,613] Trial 24 finished with value: 0.9722102488086183 and parameters: {'ncomp': 233, 'n_estimators': 699, 'max_depth': 9}. Best is trial 20 with value: 0.9754443707618992.
[I 2025-08-11 00:54:20,273] Trial 7 finished with value: 0.9619352720511269 and parameters: {'ncomp': 624, 'n_estimators': 291, 'max_depth': 2}. Best is trial 20 with value: 0.9754443707618992.
[I 2025-08-11 00:54:20,3

    AUC: 0.9932
  Fold 5


[I 2025-08-11 00:56:24,786] Trial 27 finished with value: 0.9883351862326333 and parameters: {'ncomp': 78, 'n_estimators': 185, 'max_depth': 14}. Best is trial 7 with value: 0.9956993232041504.
[I 2025-08-11 00:56:24,794] Trial 7 finished with value: 0.9956993232041504 and parameters: {'ncomp': 57, 'n_estimators': 276, 'max_depth': 17}. Best is trial 7 with value: 0.9956993232041504.
[I 2025-08-11 00:56:26,066] Trial 14 finished with value: 0.987104796083556 and parameters: {'ncomp': 164, 'n_estimators': 864, 'max_depth': 4}. Best is trial 7 with value: 0.9956993232041504.
[I 2025-08-11 00:56:26,239] Trial 26 finished with value: 0.9911717475115888 and parameters: {'ncomp': 88, 'n_estimators': 570, 'max_depth': 5}. Best is trial 7 with value: 0.9956993232041504.
[I 2025-08-11 00:56:29,570] Trial 2 finished with value: 0.9843156950322796 and parameters: {'ncomp': 528, 'n_estimators': 184, 'max_depth': 20}. Best is trial 7 with value: 0.9956993232041504.
[I 2025-08-11 00:56:30,815] Trial

    AUC: 0.9662
File updated.
File updated.
